In [2]:
!pip -q install rpy2
%load_ext rpy2.ipython


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [6]:
%%R
options(repos = c(CRAN = "https://cloud.r-project.org"))

# Install pacman if it is not already installed
if (!"pacman" %in% rownames(installed.packages())) {
  install.packages("pacman")
}

library(pacman)

# Load the packages you want
p_load(tidyverse, janitor, lubridate, readr)


In [8]:
%%R
# Month list --------------------------------------------------------------
  monthlist <- sprintf("202d", 1:12)
    y <- 2018

  # Readers (quiet & typed) -------------------------------------------------
  read_contract <- function(path) {
    read_csv(
      path,
      skip = 1,
      col_names = c(
        "contractid","planid","org_type","plan_type","partd","snp","eghp",
        "org_name","org_marketing_name","plan_name","parent_org","contract_date"
      ),
      col_types = cols(
        contractid = col_character(),
        planid     = col_double(),
        org_type   = col_character(),
        plan_type  = col_character(),
        partd      = col_character(),
        snp        = col_character(),
        eghp       = col_character(),
        org_name   = col_character(),
        org_marketing_name = col_character(),
        plan_name  = col_character(),
        parent_org = col_character(),
        contract_date = col_character()
      ),
      show_col_types = FALSE,
      progress = FALSE
    )
  }

  read_enroll <- function(path) {
    read_csv(
      path,
      skip = 1,
      col_names = c("contractid","planid","ssa","fips","state","county","enrollment"),
      col_types = cols(
        contractid = col_character(),
        planid     = col_double(),
        ssa        = col_double(),
        fips       = col_double(),
        state      = col_character(),
        county     = col_character(),
        enrollment = col_double()
      ),
      na = "*",
      show_col_types = FALSE,
      progress = FALSE
    )
  }

  # One-month loader --------------------------------------------------------
  load_month <- function(m, y) {
    c_path <- paste0(".../.../ma-data/ma/enrollment/Extracted Data/CPSC_Contract_Info_", y, "_", m, ".csv")
    e_path <- paste0(".../.../ma-data/ma/enrollment/Extracted Data/CPSC_Enrollment_Info_", y, "_", m, ".csv")

    contract.info <- read_contract(c_path) %>%
      distinct(contractid, planid, .keep_all = TRUE)   

    enroll.info <- read_enroll(e_path)

    contract.info %>%
      left_join(enroll.info, by = c("contractid","planid")) %>%
      mutate(month = as.integer(m), year = y)
  }

  # Read all months, then tidy once ----------------------------------------

  plan.year <- map_dfr(monthlist, ~ load_month(.x, y)) %>%
    arrange(contractid, planid, state, county, month) %>%
    group_by(state, county) %>%
    fill(fips, .direction = "downup") %>%                
    ungroup() %>%
    group_by(contractid, planid) %>%
    fill(plan_type, partd, snp, eghp, plan_name, .direction = "downup") %>%
    ungroup() %>%
    group_by(contractid) %>%
    fill(org_type, org_name, org_marketing_name, parent_org, .direction = "downup") %>%
    ungroup()


  # Collapse to yearly panel ------------------------------------------------
  final.plans <- plan.year %>%
    group_by(contractid, planid, fips, year) %>%
    arrange(month, .by_group = TRUE) %>%
    summarize(
      n_nonmiss        = sum(!is.na(enrollment)),
      avg_enrollment   = ifelse(n_nonmiss > 0, mean(enrollment, na.rm = TRUE), NA_real_),
      sd_enrollment    = ifelse(n_nonmiss > 1, sd(enrollment, na.rm = TRUE), NA_real_),
      min_enrollment   = ifelse(n_nonmiss > 0, min(enrollment, na.rm = TRUE), NA_real_),
      max_enrollment   = ifelse(n_nonmiss > 0, max(enrollment, na.rm = TRUE), NA_real_),
      first_enrollment = ifelse(n_nonmiss > 0, first(na.omit(enrollment)), NA_real_),
      last_enrollment  = ifelse(n_nonmiss > 0,  last(na.omit(enrollment)), NA_real_),
      state            = last(state),
      county           = last(county),
      org_type         = last(org_type),
      plan_type        = last(plan_type),
      partd            = last(partd),
      snp              = last(snp),
      eghp             = last(eghp),
      org_name         = last(org_name),
      org_marketing_name = last(org_marketing_name),
      plan_name        = last(plan_name),
      parent_org       = last(parent_org),
      contract_date    = last(contract_date),
      year             = last(year),
      .groups = "drop"
    )

Error in `map()`:
ℹ In index: 1.
Caused by error:
! '.../.../ma-data/ma/enrollment/Extracted Data/CPSC_Contract_Info_2018_202d.csv' does not exist in current working directory ('/home/sbandl2/econ470/a0/work/econ470-repo/hw1').
Run `rlang::last_trace()` to see where the error occurred.
Warning message:
In sprintf("202d", 1:12) : one argument not used by format '202d'
Error in map(.x, .f, ...) : 
Caused by error:
! '.../.../ma-data/ma/enrollment/Extracted Data/CPSC_Contract_Info_2018_202d.csv' does not exist in current working directory ('/home/sbandl2/econ470/a0/work/econ470-repo/hw1').


RInterpreterError: Failed to parse and evaluate line '# Month list --------------------------------------------------------------\n  monthlist <- sprintf("202d", 1:12)\n    y <- 2018\n\n  # Readers (quiet & typed) -------------------------------------------------\n  read_contract <- function(path) {\n    read_csv(\n      path,\n      skip = 1,\n      col_names = c(\n        "contractid","planid","org_type","plan_type","partd","snp","eghp",\n        "org_name","org_marketing_name","plan_name","parent_org","contract_date"\n      ),\n      col_types = cols(\n        contractid = col_character(),\n        planid     = col_double(),\n        org_type   = col_character(),\n        plan_type  = col_character(),\n        partd      = col_character(),\n        snp        = col_character(),\n        eghp       = col_character(),\n        org_name   = col_character(),\n        org_marketing_name = col_character(),\n        plan_name  = col_character(),\n        parent_org = col_character(),\n    